In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import datetime as dt
import seaborn as sns

## This sets the plot style
## to have a grid on a white background
sns.set_style("whitegrid")
pd.set_option('max_colwidth',100)
pd.set_option('max_columns',20)


In [2]:
Comments = pd.read_pickle("Commentsdata.pkl")
Submissions = pd.read_pickle("WSBSubmissonsdata.pkl")
print([Submissions.shape, Comments.shape])
print(Submissions.columns)
print(Comments.columns)

[(141, 10), (61515, 7)]
Index(['id', 'title', 'body', 'score', 'time_created', 'flair', 'num_comments',
       'hold_score', 'subjectivity', 'polarity'],
      dtype='object')
Index(['id', 'parent_id', 'time_created', 'body', 'score',
       'compound_sentiment', 'time_since_sub'],
      dtype='object')


In [ ]:
#Submissions.set_index(["flair", "id"]).count(level = 'flair')

In [3]:
Comments['body']=Comments['body'].astype('str')
Submissions[['title','body']]= Submissions[['title','body']].astype('str')
Comments['score']=Comments['score'].astype('float')
Submissions['score']= Submissions['score'].astype('float')
Submissions['title_len'] = Submissions.title.apply(len)
Submissions['body_len'] = Submissions.body.apply(len)
Comments['body_len'] = Comments.body.apply(len)
Submissions['date'] = pd.to_datetime(Submissions['time_created'], utc=True, unit='s')
Comments['date'] = pd.to_datetime(Comments['time_created'], utc=True, unit='s')



In [4]:
t = []
for time in Submissions['date']:
    t.append(float(time.hour*60 + time.minute))
Submissions['time'] = t

In [6]:
analyzer = SentimentIntensityAnalyzer()
cs = []
for row in range(len(Submissions)):
    if Submissions['body'].iloc[row] and Submissions['title'].iloc[row]:
        content = Submissions['title'].iloc[row] + ' ' + Submissions['body'].iloc[row]
    elif Submissions['title'].iloc[row]:
        content = Submissions['title'].iloc[row]
    elif Submissions['body'].iloc[row]:
        content = Submissions['body'].iloc[row]
    else:
        content = ''
    cs.append(analyzer.polarity_scores(content)['compound'])
Submissions['compound_score'] = cs

In [7]:
def num_early_comments(parent_id, wait_sec = 60*60):
    sub_time = Submissions.loc[parent_id,'time_created']
    return len(Comments[(Comments.parent_id == parent_id) & (Comments.time_created - sub_time < wait_sec)])

In [8]:
count = []

for row in range(len(Submissions)):
    count.append(num_early_comments(Submissions['id'].iloc[row], wait_sec = 10*60))
Submissions['num_early_comments'] = count

In [9]:
features = ['title_len','body_len','time','compound_score','num_early_comments']
popularity = ['score']

In [10]:
Submissions[features+popularity].to_csv('train_test_data.csv')